# language tools

Converting between languages, parsing languages computer languages, demonstrating tools like syntax hilighters, and outputting the diffed code


TODO: use syntax tree to convert from and to any language


use a common library for language generation 

https://bitbucket.org/birkenfeld/pygments-main/src/default/pygments/lexers/haskell.py

https://github.com/PrismJS/prism/blob/master/components/prism-markdown.js

http://qbnz.com/highlighter/

https://stackoverflow.com/questions/171924/how-do-i-syntax-check-a-bash-script-without-running-it

bash -n scriptname

Improve on the babel-preset-php pattern of replacing code with a tree, just use templates and the tree creator on regular code blocks

https://gitlab.com/kornelski/babel-preset-php/blob/master/lib/trans.js

```
function translateAssignBlock(node, scope) {
    if (isVariableCreatingAssignment(node, scope)) {
        return makeVariableDeclaration(node.left.name, "var", translateExpr(node.right, scope), node.loc, scope);
    }
    return wrapInStatement(translateAssign(node, scope));
}
```

should be something like:
```
var ASSIGMENT_EXPRESSION = (leftName, right) => {
    var leftName = right;
}

function translateAssignBlock(node, scope) {
    if(//Assignment[/Identifier].includes(node)) {
        return getAST(ASSIGMENT_EXPRESSION, node)
    }
    return getAST(STATEMENT, node)
}
```

that way when they are written in multiple languages we can transpile any language, maybe

use this library as a reference for lots of languages https://github.com/seaneshbaugh/rosetta-euler

automatically translate https://projecteuler.net/archives to any language


TODO: use babel and a transpiling kernel from babel-preset-php, 

TODO: make a generic translator that does manipulations on queries based on templates






In [ ]:
// sample definition for MatLab



In [ ]:
// definition for nodejs



## TODO: babel transpiler plugin



## syntax tree tools?




### justification

Just so it can be parsed with XPath instead. Get them all working the same!




Crap modules I won't need any more:
- css
- ASTQ
- Acorn
- esprima
- escodegen
- esquery
- jsonpath
- jsonquery
- jspath
- json:select
- jsel

TODO: get rid of jsonpath-plus, jsonstream, jspath, 



```
/* expected output `
[ Script {
    type: 'Program',
    body: [ [FunctionDeclaration], [ExpressionStatement] ],
    sourceType: 'script',
    range: [ 1, 45 ],
    comments: [],
    tokens:

<program type="program">
    <function>
    </function>
    <expression>
    </expression>
</program>

*/
```



### convert a syntax tree to html?

Reasoning. Every atom is made up of smaller component parts. This conversion creates a tree that breaks down to &lt;elements&gt; and attr="ibutes", just like neutrons and protons.


In [158]:

function statementToElement(statement) {
    var {type} = statement;
    var result = ``, attrs = ``;
    var isList = true;
    for(var i in statement) {
        if(parseInt(i) + '' === i + '') {
            result += statementToElement(statement[i])
            continue;
        }
        isList = false;
        // if the property is an Object, print out as a child
        // TODO: replace this type property when converting other trees
        //   should be like getEl() statementToElement(() => el.type)
        if(typeof statement[i].type !== 'undefined'
           || (typeof statement[i][0] !== 'undefined'
        // or if the property is an Array, print them out as child elements
           && typeof statement[i][0].type !== 'undefined')) {
            // print out the other keys just for fun matching
            attrs +=  `
${i}="" ${i}-type="${typeof statement[i]}"`;
            result += `<${i}>${statementToElement(statement[i])}</${i}>`
        } else {
            // if the property is not an object, i.e. native types
            attrs +=  `
${i}="${statement[i]}" ${i}-type="${typeof statement[i]}"`
        }
    }
    
    return isList ? result : `
<${type}${attrs}>${result}</${type}>`
}

module.exports = elementToStatement;


[Function: elementToStatement]

### convert an html tree to objects?



In [159]:

// TODO: create a pattern same as notebook markdown in core, accumulate
function accumulateComments(nodes) {
    var commentBuffer = []
    return nodes.reduce((acc, n) => {
        if(typeof n !== 'object') {
            commentBuffer.push[n];
        } else {
            n.comments = commentBuffer;
            commentBuffer = [];
            acc.push(n);
        }
        return acc;
    }, []);
}

// expects a string or a tree from JSDOM
function elementToStatement(body) {
    if(typeof body === 'string') {
        var dom = new JSDOM(body);
        body = dom.window.document.body;
    }
    if(body.nodeName === '#text') {
        return body.nodeValue;
    }
    var attrs = body.getAttributeNames()
        .filter(a => a.substr(-5) === '-type');
    if(attrs.length === 0) {
        var commentBuffer
        return Array.from(body.childNodes)
            .map(c => elementToStatement(c));
        return accumulateComments()
    }
    var result = attrs.reduce((obj, p) => {
        var childObject = Array.from(body.childNodes)
           .filter(n => n.nodeName.toLowerCase() === p.toLowerCase())[0];
        var type = body.getAttribute(p + '-type');
        console.log(type)
        if(childObject) {
            // converted from an array of childNodes above
            var children = elementToStatement(childObject)
            obj[p] = type === 'object' ? children[0] : children;
        } else {
            obj[p] = body.getAttribute(p);
        }
        return obj;
    }, {})
    return result;
}

module.exports = statementToElement;



[Function: statementToElement]

#### test tree to html?

In [161]:
var importer = require('../Core');

// TODO: make this dependency obsolete
var esquery = require('esquery');
var esprima = require('esprima'); 
var escodegen = require('escodegen');
var ASTQ  = require("astq");
var astq = new ASTQ();

var gen_options = {
    comment: true,
    tokens: true,
    whitespace: true,
    format: {indent: {style: '    '}, quotes: 'single'}
}

function testASTArray(search) {
    //var code = importer.interpret(search).code;
    var code = `
function name(params) {
    
}
console.log()
`
    var ctx = esprima.parse(code, {range: true, tokens: true,
                        comment: true, whitespace: true})
    
    // TODO: demonstrate
    console.log(astq.query(ctx, '*')[0].body);
    console.log(astq.query(ctx, '*')[0].tokens);
    console.log(esquery(ctx, '*').length);
    console.log(esquery(ctx, 'Program').length);
    
    escodegen.attachComments(ctx, ctx.comments, ctx.tokens, ctx.whitespace);
    console.log(escodegen.generate(ctx, gen_options));

    
    var page = statementToElement(ctx);
    var translated_back = elementToStatement(page)[0];
    console.log(translated_back);
    //console.log(escodegen.generate(translated_back, gen_options));

    return page;    
}

module.exports = testASTArray;

function htmlEntities(str) {
    return String(str)
        .replace(/&/g, '&amp;')
        .replace(/</g, '&lt;')
        .replace(/>/g, '&gt;')
        .replace(/"/g, '&quot;');
}

if(typeof $$ !== 'undefined') {
    var html = testASTArray('*', 'convert a tree to html')
    $$.html('<pre>' + htmlEntities(html) + '</pre>')
}


[ FunctionDeclaration {
    type: 'FunctionDeclaration',
    id:
     Identifier { type: 'Identifier', name: 'name', range: [Array] },
    params: [ [Identifier] ],
    body:
     BlockStatement { type: 'BlockStatement', body: [], range: [Array] },
    generator: false,
    expression: false,
    async: false,
    range: [ 1, 31 ] },
  ExpressionStatement {
    type: 'ExpressionStatement',
    expression:
     CallExpression {
       type: 'CallExpression',
       callee: [StaticMemberExpression],
       arguments: [],
       range: [Array] },
    range: [ 32, 45 ] } ]
[ { type: 'Keyword', value: 'function', range: [ 1, 9 ] },
  { type: 'Identifier', value: 'name', range: [ 10, 14 ] },
  { type: 'Punctuator', value: '(', range: [ 14, 15 ] },
  { type: 'Identifier', value: 'params', range: [ 15, 21 ] },
  { type: 'Punctuator', value: ')', range: [ 21, 22 ] },
  { type: 'Punctuator', value: '{', range: [ 23, 24 ] },
  { type: 'Punctuator', value: '}', range: [ 30, 31 ] },
  { type: 'Iden

<Program
type="Program" type-type="string"
body="" body-type="object"
sourceType="script" sourceType-type="string"
range="1,45" range-type="object"
comments="" comments-type="object"
tokens="" tokens-type="object"><body>
<FunctionDeclaration
type="FunctionDeclaration" type-type="string"
id="" id-type="object"
params="" params-type="object"
body="" body-type="object"
generator="false" generator-type="boolean"
expression="false" expression-type="boolean"
async="false" async-type="boolean"
range="1,31" range-type="object"><id>
<Identifier
type="Identifier" type-type="string"
name="name" name-type="string"
range="10,14" range-type="object"></Identifier></id><params>
<Identifier
type="Identifier" type-type="string"
name="params" name-type="string"
range="15,21" range-type="object"></Identifier></params><body>
<BlockStatement
type="BlockStatement" type-type="string"
body="" body-type="object"
range="23,31" range-type="object"></BlockStatement></body></FunctionDeclaration>
<ExpressionStatement
type="ExpressionStatement" type-type="string"
expression="" expression-type="object"
range="32,45" range-type="object"><expression>
<CallExpression
type="CallExpression" type-type="string"
callee="" callee-type="object"
arguments="" arguments-type="object"
range="32,45" range-type="object"><callee>
<MemberExpression
type="MemberExpression" type-type="string"
computed="false" computed-type="boolean"
object="" object-type="object"
property="" property-type="object"
range="32,43" range-type="object"><object>
<Identifier
type="Identifier" type-type="string"
name="console" name-type="string"
range="32,39" range-type="object"></Identifier></object><property>
<Identifier
type="Identifier" type-type="string"
name="log" name-type="string"
range="40,43" range-type="object"></Identifier></property></MemberExpression></callee></CallExpression></expression></ExpressionStatement></body><tokens>
<Keyword
type="Keyword" type-type="string"
value="function" value-type="string"
range="1,9" range-type="object"></Keyword>
<Identifier
type="Identifier" type-type="string"
value="name" value-type="string"
range="10,14" range-type="object"></Identifier>
<Punctuator
type="Punctuator" type-type="string"
value="(" value-type="string"
range="14,15" range-type="object"></Punctuator>
<Identifier
type="Identifier" type-type="string"
value="params" value-type="string"
range="15,21" range-type="object"></Identifier>
<Punctuator
type="Punctuator" type-type="string"
value=")" value-type="string"
range="21,22" range-type="object"></Punctuator>
<Punctuator
type="Punctuator" type-type="string"
value="{" value-type="string"
range="23,24" range-type="object"></Punctuator>
<Punctuator
type="Punctuator" type-type="string"
value="}" value-type="string"
range="30,31" range-type="object"></Punctuator>
<Identifier
type="Identifier" type-type="string"
value="console" value-type="string"
range="32,39" range-type="object"></Identifier>
<Punctuator
type="Punctuator" type-type="string"
value="." value-type="string"
range="39,40" range-type="object"></Punctuator>
<Identifier
type="Identifier" type-type="string"
value="log" value-type="string"
range="40,43" range-type="object"></Identifier>
<Punctuator
type="Punctuator" type-type="string"
value="(" value-type="string"
range="43,44" range-type="object"></Punctuator>
<Punctuator
type="Punctuator" type-type="string"
value=")" value-type="string"
range="44,45" range-type="object"></Punctuator></tokens></Program>

#### TODO test tree to html with astq

TODO: convert between astq and other tree libraries

replacing that functionality I noticed in babel somewhere



In [106]:
// reserved words

var keywords = ['type', 'id', 'params',
                'body', 'generator', 'async',
                'range', 'expression', 'callee',
                'arguments'];



expected output


```
FunctionDeclaration {
    type: 'FunctionDeclaration',
    id:
     Identifier { type: 'Identifier', name: 'name', range: [Array] },
    params: [ [Identifier] ],
    body:
     BlockStatement { type: 'BlockStatement', body: [], range: [Array] },
    generator: false,
    expression: false,
    async: false,
    range: [ 1, 31 ] },
  ExpressionStatement {
    type: 'ExpressionStatement',
    expression:
     CallExpression {
       type: 'CallExpression',
       callee: [StaticMemberExpression],
       arguments: [],
       range: [Array] },
    range: [ 32, 45 ] } ]
```


### generate a synax tree from documentation


#### test language translator

TODO: Creating an interface like this becomes a simple http request to download and create a language compatible pattern, do in C# and in Node




In [ ]:

var wireKernelInterface = {
    ...metaKernelInterface,
    // implement all requests, these are required by the meta kernel
    execute_request, inspect_request, complete_request,
    history_request, is_complete_request, shutdown_request,
    // requests maybe not required right now?
    kernel_info_request, interrupt_request, input_request,
    connect_request, comm_info_request, kernel_info_request,

    // implement all replys
    execute_reply, inspect_reply, complete_reply,
    history_reply, kernel_info_reply, is_complete_reply,
    connect_reply, comm_info_reply, kernel_info_reply,
    shutdown_reply, interrupt_reply, input_reply,

    // a few extra protocol methods
    display_data, update_display_data, execute_input,
    execute_result, error, status, clear_output,
    comm_msg, comm_close
}

module.exports = wireKernelInterface;
    